In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/private_files")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(".cache/files/chapter_three.txt")  # 과제의 문서 경로로 변경
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "다음의 컨텍스트를 기반으로 답변하세요:\n\n{context}"),
    ("human", "{question}")
])

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [ ]:
question = "Aaronson은 유죄인가요?"

result = chain.invoke(question)
print(f"질문: {question}")
print(f"답변: {result.content}")

질문: Aaronson은 유죄인가요?
답변: 아론슨(Aaronson)이 유죄인지 여부에 대해 명확한 정보가 제공되지 않았습니다. 아론슨은 조지 오웰의 소설 '1984'에서 중요한 인물 중 하나로 등장합니다. 이 소설에서는 파티의 강제적인 통제와 인간의 자유를 향한 저항이 주요 주제로 다루어지는데, 아론슨과 같은 인물들이 어떤 운명을 맞이했는지에 대한 자세한 내용이 제공되지 않습니다. 따라서 아론슨이 유죄인지 여부에 대해서는 명확한 답변을 제공할 수 없습니다.


In [9]:
question = "그가 테이블에 어떤 메시지를 썼나요?"

result = chain.invoke(question)
print(f"질문: {question}")
print(f"답변: {result.content}")

질문: 그가 테이블에 어떤 메시지를 썼나요?
답변: 그가 테이블에 쓴 메시지는 다음과 같습니다:

FREEDOM IS SLAVERY
TWO AND TWO MAKE FIVE
GOD IS POWER


In [10]:
question = "Julia는 누구인가요?"

result = chain.invoke(question)
print(f"질문: {question}")
print(f"답변: {result.content}")

질문: Julia는 누구인가요?
답변: Julia는 주인공인 윈스턴이 사랑하는 여성 캐릭터입니다. 이 문서에서는 윈스턴이 그녀를 사랑하고 그녀를 도우려는 강한 욕망을 느끼는 모습이 묘사되어 있습니다. 그러나 이 사랑과 욕망은 파티의 통제와 감시 속에서 어려움을 겪게 됩니다.
